In [7]:
!pip install tensorflowjs #model 파일 변환을 위한 tenserflowjs 설치

In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf 

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras import layers # 딥러닝 모델을 위한 라이브러리들

from sklearn.preprocessing import MultiLabelBinarizer # label 인코딩을 위한 라이브러리

from google.colab import files  #csv 파일 colab 상으로 업로드를 위함

In [9]:
seed = 10
np.random.seed(seed)

csvFile = files.upload() #데이터셋 파일 업로드


Saving pushup_dataset.csv to pushup_dataset.csv


In [ ]:
import io
data = pd.read_csv(io.BytesIO(csvFile['pushup_dataset.csv']))
data.describe()

,0_position_x,0_position_y,0_score,1_position_x,1_position_y,1_score,2_position_x,2_position_y,2_score,3_position_x,3_position_y,3_score,4_position_x,4_position_y,4_score,5_position_x,5_position_y,5_score,6_position_x,6_position_y,6_score,7_position_x,7_position_y,7_score,8_position_x,8_position_y,8_score,9_position_x,9_position_y,9_score,10_position_x,10_position_y,10_score,11_position_x,11_position_y,11_score,12_position_x,12_position_y,12_score,13_position_x,13_position_y,13_score,14_position_x,14_position_y,14_score,15_position_x,15_position_y,15_score,16_position_x,16_position_y,16_score,img_height,img_width,pose_score
count,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.000000,2951.0,2951.0,2951.000000
mean,67.082398,112.697743,0.486962,68.525568,108.561211,0.450977,64.269482,110.626124,0.353079,72.136481,106.503761,0.481744,67.342203,108.033597,0.243083,84.381934,115.275447,0.617014,75.642137,117.542207,0.515695,107.464451,138.357983,0.451485,92.054860,141.309008,0.420872,132.068919,149.831472,0.392749,128.203835,148.432537,0.396952,109.373207,136.476329,0.373709,99.473822,139.418884,0.351979,120.740680,143.589761,0.461288,104.842020,147.580222,0.432960,143.376101,168.214915,0.430920,136.461529,167.998426,0.414998,224.0,224.0,0.428027
std,66.462293,47.578275,0.357644,68.716197,48.270134,0.359785,66.160123,47.941509,0.310660,65.092386,47.435789,0.356804,65.582575,47.723498,0.249623,59.331115,45.572233,0.349137,60.616243,45.507221,0.322220,57.688904,44.367758,0.295459,60.160102,44.220865,0.298640,64.407831,43.273334,0.283065,69.932951,43.263650,0.283083,51.504398,39.522410,0.295321,53.493347,39.957060,0.293334,52.443572,34.349544,0.264820,56.188051,34.685211,0.261285,64.175317,34.028741,0.256277,68.006971,33.743777,0.252394,0.0,0.0,0.229566
min,-17.709741,-13.311756,0.000528,-9.425046,-10.793723,0.000737,-13.870014,-10.557241,0.000608,-7.927922,-15.310076,0.000386,-10.212342,-17.679083,0.000610,-9.059598,-15.425010,0.000674,-16.382549,-14.129022,0.000594,-8.927567,-6.215543,0.000537,-12.313587,-8.580183,0.000576,-13.384768,-5.287575,0.000613,-5.326314,-5.101714,0.000805,-8.109528,-6.097191,0.000802,-11.853179,-7.448987,0.000718,-10.041183,-6.990811,0.001133,-7.021356,-5.702689,0.000901,-7.155350,-0.903168,0.000942,-7.085725,-0.151686,0.000773,224.0,224.0,0.001182
25%,16.461079,79.327946,0.104750,15.748943,76.146961,0.070998,15.452414,76.649539,0.069129,20.917742,76.819516,0.109836,19.945446,75.684009,0.046651,41.350157,85.255272,0.307317,35.451662,86.626566,0.238458,54.600393,108.365130,0.218386,48.350491,108.093385,0.189169,71.484469,123.628708,0.163521,57.961259,118.547252,0.175505,86.361527,112.518501,0.138673,64.512361,114.381992,0.134153,89.246668,125.783597,0.301748,53.248761,127.481517,0.254125,95.495869,145.336560,0.226082,75.109202,145.065490,0.219557,224.0,224.0,0.270671
50%,35.715373,124.367754,0.498433,35.320292,120.087393,0.428678,33.128476,123.030995,0.270342,39.710938,116.210179,0.495700,36.281658,120.415864,0.157857,59.124682,124.772763,0.727307,50.942836,128.005098,0.545221,111.573033,144.386012,0.407696,70.689798,151.110340,0.362137,143.990113,162.993567,0.331929,136.239530,159.569307,0.348171,98.444090,144.946333,0.302250,90.694850,146.595203,0.258211,112.416912,143.974818,0.446029,100.439579,149.720996,0.419285,168.924674,172.868531,0.443134,155.521701,173.979942,0.406652,224.0,224.0,0.448038
75%,109.231764,145.571382,0.828842,115.890274,141.989171,0.7998

## 데이터 셋 설명
자체 수집한 팔굽혀펴기 데이터를 PoseNet 모델을 통해 <br>
0.코   
1.왼쪽 눈  
2.오른쪽 눈  
3.왼쪽 귀  
4.오른쪽 귀  
5.왼쪽 어깨  
6.오른쪽 어깨  
7.왼쪽 팔꿈치  
8.오른쪽 팔꿈치  
9.왼쪽 손목  
10.오른쪽 손목  
11.왼쪽 골반 부위  
12.오른쪽 골반 부위  
13.왼쪽 무릎  
14.오른쪽 무릎  
15.왼쪽 발목  
16.오른쪽 발목  

각각의 x,y 좌표와 신뢰도점수, 이미지 크기와 ,전체 신뢰도 점수를 구해 데이터 셋으로 만들었습니다.  

[출처) 포즈예측 | Tensorflow Lite](https://www.tensorflow.org/lite/examples/pose_estimation/overview?hl=ko)






In [10]:
for i in range(17):
  str_i = str(i)
  data[str_i+'_position_x'] = data[str_i+'_position_x'] / data['img_width']
  data[str_i+'_position_y'] = data[str_i+'_position_y'] / data['img_height']

# 이미지 크기로 각 x,y 좌표를 나누어 이미지 크기에 구애 받지 않는 데이터셋으로 변환
label = data['label']
features = data.drop(['label'],axis=1).drop(['img_width'],axis=1).drop(['img_height'],axis=1) # label과 feature 분리

In [11]:
features

,0_position_x,0_position_y,0_score,1_position_x,1_position_y,1_score,2_position_x,2_position_y,2_score,3_position_x,3_position_y,3_score,4_position_x,4_position_y,4_score,5_position_x,5_position_y,5_score,6_position_x,6_position_y,6_score,7_position_x,7_position_y,7_score,8_position_x,8_position_y,8_score,9_position_x,9_position_y,9_score,10_position_x,10_position_y,10_score,11_position_x,11_position_y,11_score,12_position_x,12_position_y,12_score,13_position_x,13_position_y,13_score,14_position_x,14_position_y,14_score,15_position_x,15_position_y,15_score,16_position_x,16_position_y,16_score,pose_score
0,0.091575,0.578312,0.802567,0.092882,0.542732,0.839714,0.067159,0.563080,0.588464,0.112309,0.523268,0.935588,0.098154,0.542751,0.153029,0.202142,0.598451,0.972572,0.183493,0.576958,0.679563,0.231734,0.784087,0.693994,0.230485,0.781303,0.352861,0.896258,0.770120,0.270678,0.893084,0.787690,0.271508,0.438416,0.643446,0.251209,0.292806,0.737719,0.176722,0.240550,0.793884,0.345244,0.240563,0.771910,0.390433,0.912851,0.776149,0.687812,0.912526,0.780413,0.618231,0.531187
1,0.120934,0.553329,0.532142,0.110665,0.524812,0.540048,0.103728,0.548072,0.301859,0.125859,0.498610,0.817607,0.116874,0.538948,0.368490,0.245995,0.524112,0.870264,0.196602,0.533706,0.496029,0.206697,0.744406,0.661636,0.205189,0.730808,0.461353,0.832548,0.740801,0.266246,0.817626,0.742871,0.076067,0.469243,0.658500,0.084200,0.304764,0.651139,0.147533,0.205275,0.724587,0.714054,0.201377,0.724772,0.722778,0.174527,0.900688,0.610019,0.175340,0.893444,0.639041,0.488786
2,0.116458,0.559876,0.651084,0.100523,0.530246,0.643713,0.102978,0.553798,0.429544,0.121534,0.499853,0.860058,0.108587,0.549824,0.442232,0.245023,0.519344,0.897343,0.215672,0.554685,0.512513,0.224899,0.740456,0.679680,0.205538,0.737111,0.524985,0.148076,0.698964,0.108979,0.085779,0.618855,0.081821,0.496461,0.671161,0.091583,0.302044,0.655143,0.131382,0.206007,0.729520,0.654057,0.200361,0.728449,0.648479,0.173461,0.904370,0.597678,0.175982,0.899275,0.610244,0.503846
3,0.125302,0.551466,0.800077,0.114695,0.518947,0.764695,0.108095,0.543838,0.579013,0.134185,0.483651,0.849991,0.104676,0.550948,0.543959,0.245172,0.518453,0.900969,0.204484,0.561843,0.551593,0.221288,0.730707,0.810723,0.227696,0.728486,0.463829,0.848633,0.744223,0.162366,0.163717,0.848853,0.101826,0.469076,0.660635,0.074686,0.302729,0.661020,0.128009,0.803079,0.710661,0.713964,0.200156,0.725593,0.512958,0.177312,0.888606,0.573224,0.177325,0.887476,0.569647,0.535384
4,0.116234,0.561777,0.865260,0.093222,0.536794,0.830068,0.107974,0.572191,0.297778,0.145560,0.511032,0.812742,0.105028,0.570831,0.064085,0.258436,0.518237,0.931449,0.237608,0.553966,0.672171,0.780653,0.701500,0.506313,0.242731,0.741179,0.351821,0.875144,0.725271,0.114960,0.162709,0.867205,0.058191,0.481880,0.655498,0.149651,0.412651,0.668342,0.167460,0.820088,0.710456,0.560852,0.202585,0.740656,0.697402,0.183015,0.895769,0.727983,0.178033,0.897883,0.611981,0.495304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2946,0.074436,0.711995,0.781260,0.073824,0.696010,0.703231,0.068552,0.703415,0.607264,0.081506,0.674074,0.645224,0.057250,0.706214,0.277044,0.128753,0.702313,0.687440,0.102067,0.739505,0.551849,0.650615,0.709141,0.247729,0.027027,0.808979,0.379559,0.116988,0.694532,0.353269,0.120088,0.699708,0.322349,0.425597,0.686717,0.416502,0.411258,0.681466,0.424855,0.481480,0.697574,0.636840,0.477033,0.702524,0.489723,0.883983,0.701397,0.298436,0.894568,0.685839,0.326115,0.479335
2947,0.073746,0.706463,0.797442,0.064132,0.697350,0.725700,0.066268,0.698722,0.613757,0.079904,0.672865,0.773719,0.056616,0.702743,0.315150,0.127927,0.702438,0.726808,0.062164,0.746469,0.603325,-0.000383,0.824730,0.214838,0.682565,0.726106,0.495166,0.352473,0.743486,0.274005,0.850909,0.673095,0.294198,0.422361,0.683284,0.390227,0.403411,0.676490,0.3943

In [12]:
def build_model():
  model = keras.Sequential([
                            keras.Input(shape = [len(features.keys())]), #input [,52]
                            layers.Dense(64, activation='relu'),
                            layers.Dense(64, activation='relu'),
                            layers.Dense(64, activation='relu'),
                            layers.Dense(3, activation = 'softmax') #3개의 Dense Layer와 출력 레이어로 모델 구성
  ])

  model.compile(
      loss='categorical_crossentropy',
      optimizer = 'adam',
      metrics=['acc'])
  return model

In [13]:
model = build_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                3392      
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 3)                 195       
                                                                 
Total params: 11,907
Trainable params: 11,907
Non-trainable params: 0
_________________________________________________________________


In [14]:
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(label.str.split(",")) #Label One-Hot Encoding

In [15]:
mlb.classes_ # label 출력 순서 확인

array(['nothing', 'pushuphi', 'pushupready'], dtype=object)

In [16]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(features, labels, test_size=0.3)  #Train set과 Test set 구분
train_y

array([[1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       ...,
       [1, 0, 0],
       [0, 0, 1],
       [1, 0, 0]])

In [17]:
from sklearn.utils import validation 
model.fit(train_x,train_y, epochs = 30)#모델 학습

Epoch 1/30
65/65 [==============================] - 2s 5ms/step - loss: 0.7552 - acc: 0.6964
Epoch 2/30
65/65 [==============================] - 0s 2ms/step - loss: 0.3149 - acc: 0.8949
Epoch 3/30
65/65 [==============================] - 0s 2ms/step - loss: 0.2238 - acc: 0.9206
Epoch 4/30
65/65 [==============================] - 0s 2ms/step - loss: 0.1842 - acc: 0.9370
Epoch 5/30
65/65 [==============================] - 0s 2ms/step - loss: 0.1634 - acc: 0.9438
Epoch 6/30
65/65 [==============================] - 0s 2ms/step - loss: 0.1604 - acc: 0.9492
Epoch 7/30
65/65 [==============================] - 0s 2ms/step - loss: 0.1381 - acc: 0.9540
Epoch 8/30
65/65 [==============================] - 0s 2ms/step - loss: 0.1391 - acc: 0.9535
Epoch 9/30
65/65 [==============================] - 0s 2ms/step - loss: 0.1171 - acc: 0.9617
Epoch 10/30
65/65 [==============================] - 0s 2ms/step - loss: 0.1094 - acc: 0.9651
Epoch 11/30
65/65 [==============================] - 0s 2ms/step - lo

In [18]:
model.evaluate(test_x, test_y) #테스트 데이터로 검증
mlb.classes_ # Label One-Hot Encoding부분 확인 

28/28 [==============================] - 0s 2ms/step - loss: 0.1324 - acc: 0.9605


array(['nothing', 'pushuphi', 'pushupready'], dtype=object)

In [ ]:
# 테스트를 위한 코드
# csvTestFile =  files.upload() 
# test = pd.read_csv(io.BytesIO(csvTestFile['csv_downloadfile (9).csv']))
# test
# proba = model.predict(test)
# idx = np.argmax(proba)
# proba

Save Model


In [19]:
import tensorflowjs as tfjs #저장된 모델을 불러오기 위함
tfjs.converters.save_keras_model(model,artifacts_dir="./push_up_model1.json") #tensorflow.js 사용 가능 형태로 저장
model.save('push_up_model.h5') #케라스 모델 저장